# Decision Trees Exercise
In this exercise you will show that ID3 is sub-optimal. Implement a simple version of Decision Tree, and will then apply a Decision Tree classsifier on the MNIST hand written digits dataset that we already saw.


## 1. Suboptimality of ID3
Consider the following training set, where $\mathcal{X} = \{0, 1\}^3$ and $\mathcal{Y} =\{0, 1\}$:

$$
\begin{aligned}
((1, 1, 1), 1)\\
((1, 0, 0), 1)\\
((1, 1, 0), 0)\\
((0, 0, 1), 0)
\end{aligned}
$$

Suppose we wish to use this training set in order to build a decision tree of depth 2 (i.e. for each
input we are allowed to ask two questions of the form "$x_i = 0$?" before deciding on the label).

1. Suppose we run the ID3 algorithm up to depth 2 (namely, we pick the root node and its
children according to the algorithm, but instead of keeping on with the recursion, we stop
and pick leaves according to the majority label in each subtree, once we reach depth 2).
Assume that the subroutine used to measure the quality of each feature is based on the information gain, and that if two features get the same score, one of them is picked arbitrarily.
Show that the training error of the resulting decision tree is at least 1/4.
2. Find a decision tree of depth 2, which attains zero training error.


1.
Root node error = 0.5 (because y = [1,1,0,0])
Error for split X1 = 

In [ ]:
import math
(-1) * ((2/3) * math.log2(2/3) + (1/3) * math.log2(1/3))

#### Answer
Put your answer here...

2. 
root: X1

2. A decision tree of depth 2, which attains zero training error.
$$x2\\
1 /  \quad   \backslash 0\\
x3\quad \quad \quad x1\\
1 /  \quad   \backslash 0\quad 1 /  \quad   \backslash 0\\
\textbf{1}\quad \quad \textbf{0} \quad \textbf{1} \quad\quad \textbf{0}$$

## 2. Implementing Decision Tree From Scratch
In this exercise you will need to implement a simple version of Decision Tree from scratch. Your decision tree will handle **continuous input and output** (this should actually work also for binary input attributes).

* Compelete the skeleton class below
  - `X` is a matrix of data values (rows are samples, columns are attributes)
  - `y` is a vector of corresponding target values
  - `min_leaf` is the minimal number of samples in each leaf node
  
* For splitting criterion, use either **"Train Squared Error Minimization (Reduction in Variance)"** or **"Train Absolute Error Minimization"** (choose one). Whatever you choose, make sure you implement the splitting point decision efficiently (in $O(nlgn)$ time).

* The `predict` function will use mean of the target values in the leaf node matching each row of the given `X`. The result is a vector of predictions matching the number of rows in `X`.

* To check your decision tree implementation, use the boston dataset: (https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html) split the data set into train and test using (`from sklearn.model_selection import train_test_split`)

  - Use the following to estimate what are the best hyper parameters to use for your model
```
    for min_leaf in [1,5,10,100]:
      dt = DecisionTree(X, y, n, sz, min_leaf)
      mse = # mean square error over test set
      print("min_leaf:{0} --- oob mse: {1}".format(min_leaf, mse))
```
  
  - Using your chosen hyperparameters as a final model, plot the predictions vs. true values of all the samples in the training set . Use something like:
  ```
  y_hat = dt.predict(X_train)  # forest is the chosen model
  plt.scatter(y_hat, y_test)
  ```

In [39]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

In [40]:
class DecisionTree():
    def __init__(self, X, y, min_leaf):
        self.data = X
        self.y = y
        self.min_leaf = min_leaf
        self.root = self.grow_tree(self.data, self.y)

    def grow_tree(self, X, y, depth=0):

        # check constraint
        if X.shape[0] <= self.min_leaf:
            leaf_value = np.mean(y)
            return Node(value=leaf_value)

        # find the best split
        best_feature_ixd, best_threshold = self._best_split(X, y)

        # create child nodes
        left_idxs, right_idxs = self._split(X[:, best_feature_ixd], best_threshold)

        left = self.grow_tree(X[list(left_idxs), :], y[left_idxs], depth+1)
        right = self.grow_tree(X[right_idxs, :], y[right_idxs], depth+1)
            
        return Node(best_feature_ixd, best_threshold, left, right)

    def _best_split(self, X, y):
        
        best_feature_ixd, best_threshold = None, None
        lowest_mse = self._squared_error(y)

        for feat_idx in range(X.shape[1]):
            
            # define possible thresholds for the split
            X_column = X[:, feat_idx]
            X_column_uv = np.unique(X_column)
            # thresholds = set(np.convolve(np.sort(X_column), np.ones(2)/2, mode='valid'))
            # define a step for tresholds
            tr_step = 1/len(X_column_uv)
            # define array of probabilities 
            tr_prob = np.arange(start=1, stop=(len(X_column_uv)))
            # define array of tresholds
            #print(X_column_uv)
            thresholds = np.quantile(X_column_uv, tr_prob * tr_step)
            

            for threshold in thresholds:
                # getting the left and right nodes
                left_idxs, right_idxs = self._split(X_column, threshold)

                # calculate the weighted avg. mse of children
                n = len(y)
                n_l, n_r = len(left_idxs), len(right_idxs)
                mse_l = self._squared_error(y[left_idxs]) 
                mse_r = self._squared_error(y[right_idxs])
                child_mse = (n_l/n) * mse_l + (n_r/n) * mse_r
                if best_feature_ixd is None:
                    best_feature_ixd = feat_idx
                if best_threshold is None:
                    best_threshold = threshold



                # if lowest_mse > child_mse:
                #     lowest_mse = child_mse
                #     best_feature_ixd = feat_idx
                #     best_threshold = threshold

        return best_feature_ixd, best_threshold

    def _split(self, X_column, split_thresh):        
        # get indicies after the split
        left_idxs = np.where(X_column <= split_thresh)
        right_idxs = np.where(X_column > split_thresh)

        return left_idxs[0], right_idxs[0]

    def _squared_error(self, y):
        # calculate the mean value for all observations
        y_mean = np.mean(y)

        # calculate the residuals to y_mean
        mean_squared_error = np.mean((y - y_mean)**2)

        return mean_squared_error
      

    def predict(self, X):
        return np.array([self._traverse_tree(x, self.root) for x in X])

    def _traverse_tree(self, x, node):
        if node.is_leaf_node():
            return node.value

        if x[node.feature] <= node.threshold:
            return self._traverse_tree(x, node.left)

        return self._traverse_tree(x, node.right)
  

class Node():
    def __init__(
                    self, 
                    feature=None, 
                    threshold=None, 
                    left=None, 
                    right=None, 
                    value=None
                ):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value # is it a leave node?

    def is_leaf_node(self):
        return self.value is not None



In [ ]:
# use the boston dataset
data_url = "http://lib.stat.cmu.edu/datasets/boston"
raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
target = raw_df.values[1::2, 2]

# split the data set into train and test
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=42)

In [ ]:
from sklearn.metrics import mean_squared_error

# estimate what are the best hyper parameters to use for your model
for min_leaf in [1, 5, 10, 20, 50, 100]:
      dt = DecisionTree(X_train, y_train, min_leaf)
      y_pred = dt.predict(X_test)
      mse = mean_squared_error(y_pred, y_test)
      print("min_leaf:{0} --- oob mse: {1}".format(min_leaf, mse))

The best result appears when min_leaf is equal to 10

In [ ]:
# plot the predictions vs. true values of all the samples in the training set

best_dt = DecisionTree(X_train, y_train, 10)
y_pred = best_dt.predict(X_test)
plt.scatter(x=y_pred, y=y_test, label=True, alpha=0.8)
plt.xlabel('y_pred')
plt.ylabel('y_train')

## 3. Using Decision Tree for Digits Classification
Remeber the MNIST dataset used - you will now test the power of decision trees on this problem.
This time you are given a free hand in choosing the test and train set sizes, model parameters (such as gain function and constraints over the trees) and features (whether to use binary pixel values or the original continous gray value).
  - You can use `sklearn.tree.DecisionTreeClassifier`
- Once you are satisfied with the model parameters, plot the importance of each of the pixels to the final decision.
- Last, estimate the class assignment probabilities for all the correctly classified and misclassified examples in your test data.
- Discuss your results.

In [ ]:
# code and answer go here